In [1]:
import pandas as pd
import numpy as np
from alpha_vantage.timeseries import TimeSeries

ts = TimeSeries(key='52KYU1B4HW8FCKX0')

In [2]:
class ScriptData:

    def __init__(self):
        self.fetched_data = {}
        self.converted_data = {}

    def __getitem__(self, script, datatype='converted'):
        if datatype == 'converted':
            try:
                return self.converted_data[script]
            except:
                return 'converted intraday data not available'
        elif datatype == 'fetched':
            try:
                return self.fetched_data[script]
            except:
                return

    def __setitem__(self, script, data, datatype):
        if datatype == 'fetched':
            self.fetched_data[script] = data
        elif datatype == 'converted':
            self.converted_data[script] = data

    def __contains__(self, script):
        if script in self.converted_data:
            return True
        return False

    def fetch_intraday_data(self, script):
        data, metadata = ts.get_intraday(symbol=script)
        self.__setitem__(script, data, 'fetched')

    def convert_intraday_data(self, script):
        data = self.__getitem__(script, 'fetched')
        if data is None:
            return 'fetch data for this stock first'
        df = pd.DataFrame(data).T
        df.columns = ['open', 'high', 'low', 'close', 'volume']
        df.insert(0, 'timestamp', df.index)
        df = df.astype({'open': 'float', 'high': 'float', 'low': 'float', 'close': 'float', 'volume': 'int'})
        df['timestamp'] = pd.to_datetime(df.timestamp)
        df.index = np.arange(len(data))
        self.__setitem__(script, df, 'converted')

In [3]:
script_data = ScriptData()

In [4]:
script_data.fetch_intraday_data('GOOGL')
script_data.convert_intraday_data('GOOGL')
script_data['GOOGL']

,timestamp,open,high,low,close,volume
0,2023-02-14 20:00:00,94.2400,94.24,93.500,94.1200,27980
1,2023-02-14 19:45:00,94.2700,94.32,94.200,94.2000,11286
2,2023-02-14 19:30:00,94.3400,94.34,94.250,94.3000,2459
3,2023-02-14 19:15:00,94.4500,94.50,94.260,94.2600,13234
4,2023-02-14 19:00:00,94.3000,94.49,94.300,94.3600,14382
...,...,...,...,...,...,...
95,2023-02-13 12:15:00,94.2900,94.43,94.220,94.3950,1182749
96,2023-02-13 12:00:00,94.3064,94.36,94.120,94.3000,1523977
97,2023-02-13 11:45:00,94.4600,94.50,93.980,94.3021,2155180
98,2023-02-13 11:30:00,94.6500,94.80,94.380,94.4500,1425631


In [5]:
script_data.fetch_intraday_data('AAPL')
script_data.convert_intraday_data('AAPL')
script_data['AAPL']

,timestamp,open,high,low,close,volume
0,2023-02-14 20:00:00,152.750,152.850,152.7000,152.8400,17795
1,2023-02-14 19:45:00,153.010,153.020,152.7500,152.7600,15033
2,2023-02-14 19:30:00,152.850,153.030,152.8200,153.0300,10460
3,2023-02-14 19:15:00,153.080,153.090,152.8400,152.8400,9621
4,2023-02-14 19:00:00,153.080,153.090,153.0700,153.0900,1409
...,...,...,...,...,...,...
95,2023-02-13 12:15:00,153.440,153.878,153.4100,153.8300,1507638
96,2023-02-13 12:00:00,153.510,153.690,153.2800,153.4200,1741812
97,2023-02-13 11:45:00,153.150,153.520,152.9678,153.5100,1815044
98,2023-02-13 11:30:00,152.894,153.210,152.8940,153.1423,2038315


In [6]:
'GOOGL' in script_data

True

In [7]:
'AAPL' in script_data

True

In [8]:
'NVDA' in script_data

False

In [9]:
def indicator1(df, timeperiod):
    new_df = df.loc[:,['timestamp']]
    close_series = df.loc[:, 'close']
    rolling_average = close_series.rolling(window=timeperiod, min_periods=timeperiod).mean()
    new_df.insert(1, 'indicator', rolling_average)
    return new_df

In [10]:
indicator1(script_data['GOOGL'], 5)

,timestamp,indicator
0,2023-02-14 20:00:00,NaN
1,2023-02-14 19:45:00,NaN
2,2023-02-14 19:30:00,NaN
3,2023-02-14 19:15:00,NaN
4,2023-02-14 19:00:00,94.24800
...,...,...
95,2023-02-13 12:15:00,94.48000
96,2023-02-13 12:00:00,94.45800
97,2023-02-13 11:45:00,94.40942
98,2023-02-13 11:30:00,94.37342


In [11]:
indicator1(script_data['AAPL'], 5)

,timestamp,indicator
0,2023-02-14 20:00:00,NaN
1,2023-02-14 19:45:00,NaN
2,2023-02-14 19:30:00,NaN
3,2023-02-14 19:15:00,NaN
4,2023-02-14 19:00:00,152.91200
...,...,...
95,2023-02-13 12:15:00,153.88398
96,2023-02-13 12:00:00,153.85798
97,2023-02-13 11:45:00,153.79898
98,2023-02-13 11:30:00,153.59444


In [12]:
class Strategy:

    def __init__(self, script):
        self.script = script
        self.intraday_data = pd.DataFrame()
    
    def get_script_data(self):
        script_data.fetch_intraday_data(self.script)
        script_data.convert_intraday_data(self.script)
        self.intraday_data = script_data[self.script]

    def get_signals(self):
        indicator_data = indicator1(self.intraday_data, 5)
        signals = indicator_data.loc[:, ['timestamp']]
        close_data = self.intraday_data.loc[:, 'close']
        indicator_data = indicator_data.loc[:, ['indicator']]
        signals.insert(1, 'signal', 'NO SELL')
        return signals

In [13]:
strategy = Strategy('NVDA')
strategy.get_script_data()
strategy.get_signals()

,timestamp,signal
0,2023-02-14 20:00:00,NO SELL
1,2023-02-14 19:45:00,NO SELL
2,2023-02-14 19:30:00,NO SELL
3,2023-02-14 19:15:00,NO SELL
4,2023-02-14 19:00:00,NO SELL
...,...,...
95,2023-02-13 12:15:00,NO SELL
96,2023-02-13 12:00:00,NO SELL
97,2023-02-13 11:45:00,NO SELL
98,2023-02-13 11:30:00,NO SELL
